#### This code file is same as of 1 ANN training using iterative feature selection process with base model being on common features.

#### Changes made are in this file considers all five output columns for generating a final output binary value. Other hyperparameters too ar changed

In [1]:
#Import all the necessary libraries
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 4 steps neeeded to be done for the code to be reproducible.

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as nm
nm.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# Check GPU availability
print("GPU Available:", tf.config.list_physical_devices("GPU"))

# Check CPU availability
print("CPU Available:", tf.config.list_physical_devices("CPU"))

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler 
from keras.models import Sequential #importing Sequential Model.It uses tensorflow backend and you need to have tensorflow setup. 
from keras.layers import Dense #Importing default dense layer
from sklearn.utils import shuffle
from keras.regularizers import l1, l2
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score
import time


path_dir='C:/Users/subhi/AMNS_Second_phase_ANN_model/Datasets/'
filename1='Common_Data.xlsx'
filename2='Intersection_Data.xlsx'

#Access all the necessary variables and datasets
%store -r common_data 
%store -r random_forest_data
random_forest_data.shape #161 statistical features and one output column
# Using the common_data dataframe created in ANN_Model_on_common_features
#Using the appended_data dataframe created in Random_Forest_Code 
common_data


GPU Available: []
CPU Available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


,NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN,MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN,STEELLEVELINMOLD-SETMEAN,SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN,SEG9+10+11+12BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN,LIQUIDUSTEMPMEAN,NARROWFACE1WATERFLOWMEAN,TUNDISHWEIGHTMEAN,OUTPUT
0,4.925862,4.000000,80.103621,1.177759,190.958678,1521.40790,461.735632,23.854885,1
1,4.915455,4.000000,80.104899,1.170657,195.140556,1521.76000,461.954545,23.548081,1
2,5.482414,4.000000,80.106667,1.239828,191.464138,1521.26000,461.534483,24.320115,0
3,5.471203,4.000000,80.108861,1.240000,187.756139,1521.74000,461.892405,24.275696,0
4,5.081299,4.000000,80.110282,1.190000,188.373955,1521.41000,462.028249,24.563785,1
...,...,...,...,...,...,...,...,...,...
586,2.140476,7.994048,80.108333,0.789524,213.972321,1514.56430,452.720238,23.541369,0
587,2.098624,8.052910,80.107513,0.334974,204.833228,1515.94260,443.820106,27.779471,1
588,2.097216,7.407216,80.109175,0.544330,213.607887,1512.56210,452.283505,22.814948,1
589,2.091311,7.121359,80.106019,0.722087,31.653689,1512.88320,452.150485,24.742087,1


#### Training the base model which is done in six steps 1) Splitting the data 2)Scaling the data 3)Defining the model 4)Compiling the model 5)Fit the data  6) Evaluate the model.

In [2]:
def training_data(data):
    data=shuffle(data,random_state=42)
    
    #Defining train and test sizes
    total_length= data.shape[0]
    train_size=int(data.shape[0]*0.8)
    test_size=data.shape[0]-train_size
    
    # First converting dataframe series into numpy array and setting the datatype as float32 for all data
    # In second step converting all 1D array into 2D array
    train_X=data.iloc[:,:-1][0:train_size].to_numpy().astype(nm.float32) #2D array

    train_Y=common_data['OUTPUT'][0:train_size].to_numpy().astype(nm.float32) #1D array
    train_Y=nm.reshape(train_Y,(train_Y.shape[0],1)) # 2D array

    test_X=common_data.iloc[:,:-1][train_size:].to_numpy().astype(nm.float32) #2D array

    test_Y=common_data['OUTPUT'][train_size:].to_numpy().astype(nm.float32) #1D array
    test_Y=nm.reshape(test_Y,(test_Y.shape[0],1)) # 2D array
    
    scaler=RobustScaler()
    train_X=scaler.fit_transform(train_X) 
    test_X=scaler.transform(test_X)
    
    model=Sequential() 
    model.add(Dense(50,input_dim=len(train_X[0,:]),activation='relu',kernel_regularizer=l2(0.01))) # one hidden layer
    model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer
    model.summary()
    
    model.compile(loss='binary_crossentropy',optimizer=SGD(momentum=0.9),metrics=['accuracy'])
    model.fit(train_X,train_Y,epochs=150,batch_size=len(train_X),validation_data=(test_X,test_Y))
    testPredict_probs=model.predict(test_X)
    
    # Apply threshold to obtain predicted classes
    threshold = 0.5  # Adjust this threshold as needed
    y_pred = (testPredict_probs > threshold).astype(int)

    accuracy = accuracy_score(test_Y,y_pred)
    return accuracy

accuracy=training_data(common_data)
print(accuracy)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                450       
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 501
Trainable params: 501
Non-trainable params: 0
_________________________________________________________________
Epoch 1/150
1/1 [==============================] - 0s 426ms/step - loss: 13.5324 - accuracy: 0.5042 - val_loss: 0.8589 - val_accuracy: 0.5462
Epoch 2/150
1/1 [==============================] - 0s 29ms/step - loss: 5.6168 - accuracy: 0.4788 - val_loss: 0.8669 - val_accuracy: 0.5630
Epoch 3/150
1/1 [==============================] - 0s 27ms/step - loss: 4.3841 - accuracy: 0.5021 - val_loss: 0.8437 - val_accuracy: 0.5462
Epoch 4/150
1/1 [===============

1/1 [==============================] - 0s 25ms/step - loss: 0.9138 - accuracy: 0.5339 - val_loss: 0.9012 - val_accuracy: 0.6134
Epoch 55/150
1/1 [==============================] - 0s 26ms/step - loss: 0.9110 - accuracy: 0.5381 - val_loss: 0.9015 - val_accuracy: 0.5882
Epoch 56/150
1/1 [==============================] - 0s 25ms/step - loss: 0.9085 - accuracy: 0.5318 - val_loss: 0.9023 - val_accuracy: 0.5462
Epoch 57/150
1/1 [==============================] - 0s 22ms/step - loss: 0.9063 - accuracy: 0.5339 - val_loss: 0.9034 - val_accuracy: 0.5378
Epoch 58/150
1/1 [==============================] - 0s 22ms/step - loss: 0.9044 - accuracy: 0.5360 - val_loss: 0.9048 - val_accuracy: 0.5462
Epoch 59/150
1/1 [==============================] - 0s 25ms/step - loss: 0.9028 - accuracy: 0.5487 - val_loss: 0.9062 - val_accuracy: 0.5546
Epoch 60/150
1/1 [==============================] - 0s 23ms/step - loss: 0.9014 - accuracy: 0.5466 - val_loss: 0.9075 - val_accuracy: 0.5462
Epoch 61/150
1/1 [========

1/1 [==============================] - 0s 23ms/step - loss: 0.8489 - accuracy: 0.5678 - val_loss: 0.8576 - val_accuracy: 0.5714
Epoch 113/150
1/1 [==============================] - 0s 22ms/step - loss: 0.8481 - accuracy: 0.5699 - val_loss: 0.8570 - val_accuracy: 0.5714
Epoch 114/150
1/1 [==============================] - 0s 22ms/step - loss: 0.8474 - accuracy: 0.5678 - val_loss: 0.8563 - val_accuracy: 0.5714
Epoch 115/150
1/1 [==============================] - 0s 24ms/step - loss: 0.8466 - accuracy: 0.5678 - val_loss: 0.8556 - val_accuracy: 0.5714
Epoch 116/150
1/1 [==============================] - 0s 23ms/step - loss: 0.8459 - accuracy: 0.5699 - val_loss: 0.8549 - val_accuracy: 0.5714
Epoch 117/150
1/1 [==============================] - 0s 24ms/step - loss: 0.8451 - accuracy: 0.5699 - val_loss: 0.8542 - val_accuracy: 0.5630
Epoch 118/150
1/1 [==============================] - 0s 23ms/step - loss: 0.8444 - accuracy: 0.5699 - val_loss: 0.8536 - val_accuracy: 0.5630
Epoch 119/150
1/1 [=

NameError: name 'base_model' is not defined

In [ ]:
# Create a list to store the selected features and their corresponding accuracies
base_features = common_data.columns.tolist()[:-1]
base_features

new_accuracy=training_data(base_features,common_data)

In [ ]:
print(new_accuracy)
base_features

In [ ]:
new_data=common_data.loc[:,base_features]
new_data

### Data for new model

In [ ]:
%store -r feature_imp_RF 
#using it from Random_Forest_Code
#it is a dataframe which stores the features and their corresponding importance score in descending order
total_features=feature_imp_RF["Feature"]
total_features_list= total_features.tolist()

remaining_features=[]
for feature in total_features_list:
    if feature not in base_features:
        remaining_features.append(feature) 
        
print(len(remaining_features))
#feature_imp_RF["Feature"]

### Iterating over remaining features and training at each iteration

In [ ]:
new_features=[]
optimal_features=[]
accuracy_store=[base_accuracy]

# Start the timer
start_time = time.time()

for feature in remaining_features:
    # Add one feature to the feature set
    new_features = base_features + [feature]
    
    #Creating a new data and shuffling the data
    new_data=random_forest_data.loc[:,new_features]
    new_data = shuffle(new_data, random_state=42)
    
    #1 Splitting the new_data into train and test
    new_total_length= new_data.shape[0]
    new_train_size=int(new_data.shape[0]*0.8)
    new_test_size=new_data.shape[0]-new_train_size
    
    new_train_X=new_data.iloc[:,:][0:new_train_size].to_numpy().astype(nm.float32) #2D array
    new_train_Y=random_forest_data['OUTPUT'][0:new_train_size].to_numpy().astype(nm.float32) #1D array
    new_train_Y=nm.reshape(new_train_Y,(new_train_Y.shape[0],1)) # 2D array
    
    new_test_X=new_data.iloc[:,:][new_train_size:].to_numpy().astype(nm.float32) #2D array
    new_test_Y=random_forest_data['OUTPUT'][new_train_size:].to_numpy().astype(nm.float32) #1D array
    new_test_Y=nm.reshape(new_test_Y,(new_test_Y.shape[0],1)) # 2D array}

    #2 Scaling the data
    scaler=RobustScaler()
    new_train_X=scaler.fit_transform(new_train_X) 
    new_test_X=scaler.transform(new_test_X)
    
    #3 Create a new model with the updated feature set
    new_model=Sequential() 
    new_model.add(Dense(50,input_dim=len(new_features),activation='relu',kernel_regularizer=l2(0.01))) # one hidden layer
    new_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer
        
    #4 Compile the new model
    new_model.compile(loss='binary_crossentropy',optimizer=SGD(momentum=0.9),metrics=['accuracy'])
    
    print(feature)
    
    #5 Train the new model
    new_model.fit(new_train_X,new_train_Y,epochs=150,batch_size=len(new_train_X),validation_data=(new_test_X,new_test_Y))
    
    #6 Evaluate the new model
    testPredict_probs_new=new_model.predict(new_test_X)
    threshold = 0.5  # Adjust this threshold as needed
    y_new_pred = (testPredict_probs_new > threshold).astype(int)
    new_accuracy = accuracy_score(new_test_Y,y_new_pred)
    accuracy_store.append(new_accuracy)
    
    # Update the baseline feature set if the accuracy score improved
    if new_accuracy> base_accuracy:
        base_features = new_features
        base_accuracy = new_accuracy
        optimal_features.append(new_features)
    
    # Check for convergence
    #if len(base_features) == random_forest_data.iloc[:,:-1].shape[1]:
     #   break
        
# Stop the timer and calculate the elapsed time
end_time = time.time()
training_time = end_time - start_time

In [ ]:
print("Simulation ended",training_time)
print("Best feature set:", base_features)
print("Best accuracy score:", base_accuracy)

In [ ]:
accuracy_store

In [ ]:
common_accuracy_store=accuracy_store
common_base_features=base_features
common_base_accuracy=base_accuracy
common_optimal_features=optimal_features

%store common_accuracy_store
%store common_base_features
%store common_base_accuracy
%store common_optimal_features

In [ ]:
len(new_train_X)

In [ ]:
type(new_data)